## **[2024년 1학기 로봇공학] Ch. 4-c. Introduction to Swift**

- 작성자 : 서울시립대학교 전자전기컴퓨터공학부 박경훈 교수(gyunghoon.park@uos.ac.kr)
- 작성일 : 2024. 4. 23.

In [1]:
import spatialgeometry as sg
import spatialmath as sm
import roboticstoolbox as rtb
import numpy as np
from spatialmath.base import *
from math import *
from scipy import *

Ch. 4-c에서는 `Robotics Toolbox for Python` 상에서 수행할 수 있는 간단한 시뮬레이션 도구인 `Swift`에 대해 간단히 소개합니다.\
(github repo. : https://github.com/jhavl/swift)\
만약 `Swift`가 설치되지 않았다면 `pip`를 이용하여 추가로 설치해줄 필요가 있습니다. 설치가 마무리되었다면 아래와 같이 불러옵니다.

In [2]:
from roboticstoolbox.backends import swift

### **Topic 1 : 개요**

기본적으로 `Swift`는 웹 브라우저 기반의 시뮬레이터이며, 아래와 같이 실행시키면 현재 플랫폼에서 사용하는 웹 브라우저의 새 탭에 시뮬레이션 환경을 띄웁니다.

In [3]:
env = swift.Swift()
env.launch()

만약 Jupyter notebook 안에서 실행시키고 싶다면, 아래와 같이 `brower="notebook"`이라는 옵션을 추가하면 됩니다.

In [4]:
env.launch(realtime=True, browser="notebook")

  **혹시 Swift 환경이 제대로 작동하지 않는다면, 아래 내용을 참고해주세요.**
- (Windows 환경에서) Swift가 제대로 동작하지 않는 경우에는, Swift Library 내의 SwiftRoute.py의 특정 경로를 새로 지정해주서야 합니다. 
- swift 파일 위치는 print 함수를 통해 얻을 수 있습니다.
- 자세한 사항은 아래 링크의 내용을 확인해보시기 바랍니다. (https://github.com/petercorke/robotics-toolbox-python/issues/383)

In [5]:
print(swift)

<module 'roboticstoolbox.backends.swift' from 'c:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\roboticstoolbox\\backends\\swift\\__init__.py'>


앞서 소개한 대로 Swift 환경에서는 URDF 모델을 불러오고 시뮬레이션할 수 있습니다.

In [6]:
panda = rtb.models.URDF.Panda()
panda.plot(panda.qr)

Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link0.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link1.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link2.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link3.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link4.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link5.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/me

Swift backend, t = 0.05, scene:
  panda

### **Topic 2 : Collision Detection**

Robotics에서 활용하는 여러 시뮬레이터들과 유사하게, `Swift` 역시 로봇과 객체 사이의 충돌 감지 알고리즘을 제공합니다.\
이를 확인하기 위해 다음의 환경을 설정합시다.

In [7]:
env = panda.plot(panda.qr, backend="swift")
from spatialgeometry import Cuboid
box = Cuboid([1, 1, 1], pose=sm.SE3.Tx(1.1))

env.add(box) # add box to graphics
env.step() # update the graphics

Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link0.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link1.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link2.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link3.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link4.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link5.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/me

현재 로봇과 박스 사이에는 접촉이 없는 것을 눈으로 확인할 수 있습니다.\
이제 `iscollided` 함수를 이용하여, 이 사실을 출력해보도록 합시다.\
(위 기능을 사용하기 위해서는 `pybullet` 패키지가 필요합니다.)

In [8]:
panda.iscollided(panda.qr, box)

False

Box의 위치를 조금 수정하면 다른 결과가 나옴을 아래와 같이 확인할 수 있습니다.

In [9]:
env2 = panda.plot(panda.qr, backend="swift")
from spatialgeometry import Cuboid
box2 = Cuboid([1, 1, 1], pose=sm.SE3.Tx(1))

env2.add(box2) # add box to graphics
env2.step() # update the graphics

Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link0.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link1.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link2.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link3.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link4.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link5.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/me

True

In [ ]:
panda.iscollided(panda.qr, box2)

### **Topic 3 : Resolved-rate Motion Control**

아래는 github repo.(https://github.com/jhavl/swift)에서 제공하는 panda의 motion control 시뮬레이션 예제입니다.\
사용한 주요 함수들은 아래와 같습니다. 
- `fkine` : 로봇의 forward kinematics를 계산해주는 함수입니다. 
- `p_servo` : 로봇의 end-effector를 특정 configuration으로 이동하게 해주는 end-effector velocity를 생성합니다. (제어 관련 함수)
- `pinv` : 어떤 행렬의 Pseudo-inverse를 출력하는 함수입니다.

In [12]:
import spatialgeometry as sg

# Make and instance of the Swift simulator and open it
env = swift.Swift()
env.launch(realtime=True, browser="notebook")

panda.q = panda.qr

# Set a desired and effector pose an an offset from the current end-effector pose
Tep = panda.fkine(panda.q) * sm.SE3.Tx(0.2) * sm.SE3.Ty(0.2) * sm.SE3.Tz(0.3)
axes = sg.Axes(length=0.3, base=Tep)
env.add(axes)

# Add the robot to the simulator
env.add(panda)

# Simulate the robot while it has not arrived at the goal
arrived = False

while not arrived:
    # Work out the required end-effector velocity to go towards the goal
    v, arrived = rtb.p_servo(panda.fkine(panda.q), Tep, gain = 1, threshold = 0.02)
    
    # Set the Panda's joint velocities
    panda.qd = np.linalg.pinv(panda.jacobe(panda.q)) @ v
    
    # Step the simulator by 1 milliseconds
    env.step(0.001) 


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\spatialgeometry\geom\Shape.py:81: FutureWarning: base kwarg is deprecated, use pose instead
  warn("base kwarg is deprecated, use pose instead", FutureWarning)


Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link0.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link1.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link2.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link3.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link4.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/meshes/visual/link5.dae
Retrieving file: C:/Users/user/AppData/Local/Programs/Python/Python312/Lib/site-packages/rtbdata/xacro/franka_description/me

AttributeError: 'Swift' object has no attribute 'record_start'